In [3]:
cd ..

/home/avani.gupta/bpirl2


In [4]:
from utils import calc_third_quartile, get_unique_act, get_compliant_cases 
import numpy as np
import os
import pickle
import pandas as pd
import random
from statistics import mean, median

In [4]:
dataset = "helpdesk"
df2 = pd.read_pickle('dataset/preprocessed/'+dataset+'_design_mat.pkl')

In [5]:
graphs = {}
graph = {}
graph[-1] = {1:3644, 3:108,-1:100}
graph[0] = {1:3644, 3:108,0:100}  #end of process instance 1, start of instance 2
graph[3] = {1:105}
graph[1] = {8:3483,1:394}
graph[8] = {2:42, 9:851, 6:4150,4:9, 7:4} #edges:weights
graph[2] = {5:3, 6:37}
graph[9] = {6:382,8:4270}
graph[4] = {6:8}
graph[7] = {6:3}
graph[5] = {6:4}
graph[6] = {0:3804} #end
graphs['helpdesk'] = graph

In [6]:
# Python program to print all paths from a source to destination.
  
from collections import defaultdict
  
# This class represents a directed graph
# using adjacency list representation
class Graph:
  
    def __init__(self, vertices):
        # No. of vertices
        self.V = vertices
         
        # default dictionary to store graph
        self.graph = defaultdict(list)
  
    # function to add an edge to graph
    def addEdge(self, u, v):
        self.graph[u].append(v)
  
    '''A recursive function to print all paths from 'u' to 'd'.
    visited[] keeps track of vertices in current path.
    path[] stores actual vertices and path_index is current
    index in path[]'''
    def printAllPathsUtil(self, u, d, visited, path):
 
        # Mark the current node as visited and store in path
        visited[u]= True
        path.append(u)
 
        # If current vertex is same as destination, then print
        # current path[]
        if u == d:
            print(path)
        else:
            # If current vertex is not destination
            # Recur for all the vertices adjacent to this vertex
            for i in self.graph[u]:
                if visited[i]== False:
                    self.printAllPathsUtil(i, d, visited, path)
                     
        # Remove current vertex from path[] and mark it as unvisited
        path.pop()
        visited[u]= False
  
  
    # Prints all paths from 's' to 'd'
    def printAllPaths(self, s, d):
 
        # Mark all the vertices as not visited
        visited =[False]*(self.V)
 
        # Create an array to store paths
        path = []
 
        # Call the recursive helper function to print all paths
        self.printAllPathsUtil(s, d, visited, path)
  


In [7]:
g = Graph(10)
for gg in graph:
    for dest in graph[gg]:
        g.addEdge(gg,dest)

In [8]:
s = -1
d = 0
print ("Following are all different paths from % d to % d :" %(s, d))

g.printAllPaths(s, d)

Following are all different paths from -1 to  0 :
[-1, 1, 8, 2, 5, 6, 0]
[-1, 1, 8, 2, 6, 0]
[-1, 1, 8, 6, 0]
[-1, 1, 8, 4, 6, 0]
[-1, 1, 8, 7, 6, 0]
[-1, 3, 1, 8, 2, 5, 6, 0]
[-1, 3, 1, 8, 2, 6, 0]
[-1, 3, 1, 8, 6, 0]
[-1, 3, 1, 8, 4, 6, 0]
[-1, 3, 1, 8, 7, 6, 0]


In [10]:
paths = [[-1, 1, 8, 2, 5, 6, 0],
[-1, 1, 8, 2, 6, 0],
[-1, 1, 8, 6, 0],
[-1, 1, 8, 4, 6, 0],
[-1, 1, 8, 7, 6, 0],
[-1, 3, 1, 8, 2, 5, 6, 0],
[-1, 3, 1, 8, 2, 6, 0],
[-1, 3, 1, 8, 6, 0],
[-1, 3, 1, 8, 4, 6, 0],
[-1, 3, 1, 8, 7, 6, 0]]

In [18]:
gs_lis = [[1, 8, 6], [3, 1, 8, 6]]

In [19]:
# get process flow compliance cases only
df = get_compliant_cases(df2,dset=dataset)

In [21]:
if [1, 8, 6] in gs_lis:
    print("yes")

yes


In [25]:
cases_sat = []
dat_group = df.groupby("CaseID")
for name, gr in dat_group:
    act = list(gr['class'])
    if act in gs_lis:
        cases_sat.append(name)

In [27]:
len(cases_sat)

2243

In [31]:
len(cases_sat)/len(dat_group)

0.7107097591888466

In [36]:
df = df.loc[df['CaseID'].isin(cases_sat)]

In [37]:
dat_group = df.groupby("CaseID")

total_iter = len(dat_group.ngroup())
case_duration_dic = {}
for name, gr in dat_group:
    case_duration_dic[name] = gr['duration_time'].sum()  

In [38]:
max(df['duration_time'])

46.7043287037037

In [39]:
mean(case_duration_dic.values())

5.984576341416093

In [44]:
# # quartile calculation
# import statistics
# def calc_third_quartile(lis):
#     lis.sort()
#     size = len(lis)
#     lis_upper_half = lis[size//2:-1]
#     third_quartile = statistics.median(lis_upper_half)
#     return third_quartile

# case_durations = list(case_duration_dic.values())
# third_quartile  = calc_third_quartile(case_durations)

In [45]:
third_quartile = 13.90056712962963

In [46]:
cases_gs = []
cases_gv = []
for k,v in case_duration_dic.items():
    if v <= third_quartile:
        cases_gs.append(k)
    else:
        cases_gv.append(k)

In [48]:
len(cases_gs), len(cases_gv)

(1894, 349)

In [49]:
tot = len(cases_gs)+ len(cases_gv)
percent_gs_cases = len(cases_gs) / tot
print(percent_gs_cases)

0.8444048149799376


In [50]:
cases_train = cases_gs
cases_test = cases_gv
data_train = df.loc[df['CaseID'].isin(cases_train)]
data_test = df.loc[df['CaseID'].isin(cases_test)]

In [51]:
a = get_unique_act(data_train)

In [52]:
len(a)

5

In [53]:
tot = get_unique_act(df)
len(tot)

5

In [54]:
lis = []
for act in tot:
    if act not in a:
        lis.append(act)
lis   

[]

In [55]:
for act in lis:
    df_sub = df[df["class"] == act]
    caseid_lis = list(df_sub["CaseID"])
    l = len(caseid_lis)
    caseid_sel = caseid_lis[:l//2]
    if len(caseid_sel) == 0:
        caseid_sel = caseid_lis
        
    r = df.loc[df['CaseID'].isin(caseid_sel)]
    data_train = data_train.append(r)


In [56]:
data_train

,0,1,2,3,4,5,6,7,8,9,duration_time,remaining_time,class,CaseID
0,0,1,0,0,0,0,0,0,0,0,0.000000,2.014051,1,2
1,0,0,0,0,0,0,0,0,1,0,0.000174,2.013877,8,2
2,0,0,0,0,0,0,1,0,0,0,2.013877,0.000000,6,2
3,0,1,0,0,0,0,0,0,0,0,0.000000,5.296655,1,3
4,0,0,0,0,0,0,0,0,1,0,5.293113,0.003542,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13697,0,0,0,0,0,0,0,0,1,0,0.000220,0.049086,8,4576
13698,0,0,0,0,0,0,1,0,0,0,0.049086,0.000000,6,4576
13704,0,1,0,0,0,0,0,0,0,0,0.000000,0.145556,1,4579
13705,0,0,0,0,0,0,0,0,1,0,0.145417,0.000139,8,4579


In [57]:
len(get_unique_act(data_train)), len(get_unique_act(data_test))

(5, 5)

In [59]:
name = dataset+'_d3'
pickle.dump(data_train, open(name+"_train_RL.pkl", "wb"))
pickle.dump(data_test, open(name+"_test_RL.pkl", "wb"))

In [60]:
pd.read_pickle(name+"_train_RL.pkl")

,0,1,2,3,4,5,6,7,8,9,duration_time,remaining_time,class,CaseID
0,0,1,0,0,0,0,0,0,0,0,0.000000,2.014051,1,2
1,0,0,0,0,0,0,0,0,1,0,0.000174,2.013877,8,2
2,0,0,0,0,0,0,1,0,0,0,2.013877,0.000000,6,2
3,0,1,0,0,0,0,0,0,0,0,0.000000,5.296655,1,3
4,0,0,0,0,0,0,0,0,1,0,5.293113,0.003542,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13697,0,0,0,0,0,0,0,0,1,0,0.000220,0.049086,8,4576
13698,0,0,0,0,0,0,1,0,0,0,0.049086,0.000000,6,4576
13704,0,1,0,0,0,0,0,0,0,0,0.000000,0.145556,1,4579
13705,0,0,0,0,0,0,0,0,1,0,0.145417,0.000139,8,4579


In [ ]:
pd.read_pickle("helpdesk_3_train_RL.pkl")

In [13]:
dataset = 'helpdesk'
name = dataset+'_d2'
df2 = pd.read_pickle('dataset/preprocessed/'+name+"_test_RL.pkl")
gs_lis = [[1, 8, 6], [3, 1, 8, 6]]

In [15]:
cases_sat = []
dat_group = df2.groupby("CaseID")
for name, gr in dat_group:
    act = list(gr['class'])
    if act in gs_lis:
        cases_sat.append(name)

In [17]:
len(cases_sat)/len(df2['CaseID'].unique())

0.6797468354430379

In [12]:
data_filtered = df2.loc[df2['CaseID'].isin(cases_sat)]

In [18]:
name = dataset+'_d2'
pickle.dump(data_filtered, open(name+"_test_RL_filtered.pkl", "wb"))